> Note: If you have a different iSamples PQG parquet file from another provider, set `file_url` and `LOCAL_PATH` accordingly. All queries below will still work because they rely on PQG structure and iSamples model semantics.

# iSamples PQG Parquet Analysis (using OpenContext dataset)

This notebook analyzes an iSamples Property Graph (PQG) parquet file. The sample file we use happens to be produced from OpenContext, but the schema, node types, and graph patterns are iSamples‑generic.

## Key Distinction: PQG framework vs iSamples model vs provider data

We’ll keep these layers straight:

1. Generic PQG (Property Graph) framework
   - Core graph fields: `s` (subject), `p` (predicate), `o` (object array), `n` (graph name)
   - Edges are rows with `otype = '_edge_'`
   - Graph traversal patterns (joins on s/p/o) are domain‑agnostic

2. iSamples metadata model (provider‑agnostic domain schema)
   - Entity types: `MaterialSampleRecord`, `SamplingEvent`, `GeospatialCoordLocation`, `SamplingSite`, `IdentifiedConcept`, `Agent`, etc.
   - Predicates like `produced_by`, `sample_location`, `sampling_site`, `has_material_category`, etc.
   - These are defined by the iSamples model, not specific to OpenContext

3. Provider data (e.g., OpenContext)
   - A particular provider’s content fills the iSamples model
   - The dataset URL we load is from OpenContext, but the analysis is reusable for any iSamples PQG parquet

## Setup and Data Loading

In [ ]:
import duckdb
import pandas as pd
import numpy as np
from pathlib import Path
import urllib.request
import os

# Configuration
file_url = "https://storage.googleapis.com/opencontext-parquet/oc_isamples_pqg.parquet"
LOCAL_PATH = "/Users/raymondyee/Data/iSample/oc_isamples_pqg.parquet"

In [ ]:
# Check if local file exists, download if not
if not os.path.exists(LOCAL_PATH):
    print(f"Local file not found at {LOCAL_PATH}")
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(LOCAL_PATH), exist_ok=True)
    
    print(f"Downloading {file_url} to {LOCAL_PATH}...")
    urllib.request.urlretrieve(file_url, LOCAL_PATH)
    print("Download completed!")
else:
    print(f"Local file already exists at {LOCAL_PATH}")

# Use local path for parquet operations
parquet_path = LOCAL_PATH
print(f"Using parquet file: {parquet_path}")

## Understanding the Data Structure

### PQG framework (generic)
The parquet file uses a property graph model where both entities (nodes) and relationships (edges) are stored in one table. This pattern is generic and reusable across providers.

Core PQG fields:
- `s` (subject): source node row_id for an edge
- `p` (predicate): relationship type
- `o` (object): array of target row_ids
- `n` (name): graph context/namespace (often null)

Edges are rows with `otype = '_edge_'`.

### iSamples metadata model (provider‑agnostic)
Values in `otype` and `p` map to the iSamples domain schema, independent of the specific provider:
- Entity types: `MaterialSampleRecord`, `SamplingEvent`, `GeospatialCoordLocation`, `SamplingSite`, `IdentifiedConcept`, `Agent`, `_edge_`
- Common predicates: `produced_by`, `sample_location`, `sampling_site`, `site_location`, `has_material_category`, `has_responsibility_actor`, etc.

We’ll demonstrate queries that traverse the generic PQG structure while filtering/labeling using the iSamples model.

Note: The example parquet we load is produced from OpenContext content, but the analysis patterns apply to any iSamples PQG parquet.

In [ ]:
# Create a DuckDB connection
conn = duckdb.connect()

# Create view for the parquet file
conn.execute(f"CREATE VIEW pqg AS SELECT * FROM read_parquet('{parquet_path}');")

# Count records
result = conn.execute("SELECT COUNT(*) FROM pqg;").fetchone()
print(f"Total records: {result[0]:,}")

In [ ]:
# Schema information
print("Schema information:")
schema_result = conn.execute("DESCRIBE pqg;").fetchall()
for row in schema_result[:10]:  # Show first 10 columns
    print(f"{row[0]:25} | {row[1]}")
print(f"... and {len(schema_result) - 10} more columns")

In [ ]:
# Examine the distribution of entity types (iSamples model types)
entity_stats = conn.execute("""
    SELECT
        otype,
        COUNT(*) as count,
        COUNT(DISTINCT pid) as unique_pids,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
    FROM pqg
    GROUP BY otype
    ORDER BY count DESC
""").fetchdf()

print("Entity Type Distribution (iSamples model types):")
print(entity_stats)

### Graph structure fields (PQG)

The fields `s`, `p`, `o`, `n` are part of the generic PQG representation:
- s (subject): row_id of the source entity
- p (predicate): relationship type
- o (object): array of target row_ids
- n (name): graph context (usually null)

These patterns are provider‑agnostic. The iSamples model provides the semantics for common predicates such as:
- Sample (s) produced_by (p) SamplingEvent (o)
- Event (s) sample_location (p) GeospatialCoordLocation (o)

In [ ]:
# Explore edge predicates (iSamples model predicates)
edge_predicates = conn.execute("""
    SELECT
        p as predicate,
        COUNT(*) as usage_count,
        COUNT(DISTINCT s) as unique_subjects
    FROM pqg
    WHERE otype = '_edge_'
    GROUP BY p
    ORDER BY usage_count DESC
    LIMIT 15
""").fetchdf()

print("Most common relationship types (iSamples predicates):")
print(edge_predicates)

## Practical Query Examples

The following queries demonstrate both:
1. **Generic PQG patterns**: How to traverse graphs using s/p/o relationships
2. **OpenContext specifics**: The actual entity types and predicates for archaeological data

### Graph walk patterns used

We use two traversal paths to attach coordinates to samples:

- Direct event location:
  Sample → produced_by → SamplingEvent → sample_location → GeospatialCoordLocation
- Site-derived location:
  Sample → produced_by → SamplingEvent → sampling_site → SamplingSite → site_location → GeospatialCoordLocation

### Query 1: Find Samples with Geographic Coordinates

This query demonstrates:
- **Generic PQG pattern**: Multi-hop graph traversal through edges
- **OpenContext specifics**: Archaeological entity types and relationships

In [ ]:
# Find samples with geographic coordinates (via SamplingEvent)
# PQG: traverse edges by joining on s/p/o; iSamples: filter types/predicates

# Ensure we have a working connection
try:
    conn.execute("SELECT 1").fetchone()
except:
    conn = duckdb.connect()
    conn.execute(f"CREATE VIEW pqg AS SELECT * FROM read_parquet('{parquet_path}');")

samples_with_coords = conn.execute("""
    SELECT
        s.pid as sample_id,
        s.label as sample_label,
        s.description,
        g.latitude,
        g.longitude,
        g.place_name,
        'direct_event_location' as location_type
    FROM pqg s
    JOIN pqg e1   ON s.row_id = e1.s AND e1.p = 'produced_by'
    JOIN pqg evt  ON e1.o[1] = evt.row_id
    JOIN pqg e2   ON evt.row_id = e2.s AND e2.p = 'sample_location'
    JOIN pqg g    ON e2.o[1] = g.row_id
    WHERE s.otype = 'MaterialSampleRecord'
      AND evt.otype = 'SamplingEvent'
      AND g.otype = 'GeospatialCoordLocation'
      AND g.latitude IS NOT NULL
    LIMIT 100
""").fetchdf()

print(f"Found {len(samples_with_coords)} samples with direct event coordinates")
samples_with_coords.head()

### Using Ibis for Cleaner Multi-Step Joins

Ibis provides a more Pythonic interface for the same **generic PQG graph traversal patterns**, while making **OpenContext-specific** entity filtering clearer.

In [ ]:
# Import Ibis for cleaner data manipulation
import ibis
from ibis import _

ibis.options.interactive = True

# Create Ibis connection using DuckDB
ibis_conn = ibis.duckdb.connect()

# Register the parquet file as a table in Ibis
pqg = ibis_conn.read_parquet(parquet_path, table_name='pqg')

print("Ibis setup complete!")
print(f"Table columns: {pqg.columns}")
print(f"Total records: {pqg.count().execute():,}")

In [ ]:
# Ibis version: Find samples with geographic coordinates through SamplingEvent

# Base tables with iSamples model type filters
samples = pqg.filter(_.otype == 'MaterialSampleRecord').alias('samples')
events = pqg.filter(_.otype == 'SamplingEvent').alias('events')
locations = pqg.filter(_.otype == 'GeospatialCoordLocation').alias('locations')
edges = pqg.filter(_.otype == '_edge_').alias('edges')

# Sample -> produced_by -> SamplingEvent
sample_to_event = (
    samples
    .join(
        edges.filter(_.p == 'produced_by'),
        samples.row_id == edges.s
    )
    .join(
        events,
        edges.o[0] == events.row_id
    )
)

# SamplingEvent -> sample_location -> GeospatialCoordLocation
location_edges = edges.filter(_.p == 'sample_location').alias('location_edges')
event_to_location = (
    sample_to_event
    .join(
        location_edges,
        events.row_id == location_edges.s
    )
    .join(
        locations.filter(_.latitude.notnull()),
        location_edges.o[0] == locations.row_id
    )
)

samples_with_coords_ibis = (
    event_to_location
    .select(
        sample_id=samples.pid,
        sample_label=samples.label,
        description=samples.description,
        latitude=locations.latitude,
        longitude=locations.longitude,
        place_name=locations.place_name,
        location_type=ibis.literal('direct_event_location')
    )
    .limit(100)
)

result_ibis = samples_with_coords_ibis.execute()
print(f"Found {len(result_ibis)} samples with direct event coordinates (Ibis)")
result_ibis.head()

In [ ]:
# Ibis version: Find samples via site location path

sites = pqg.filter(_.otype == 'SamplingSite').alias('sites')

# Define edge tables
event_edges = edges.filter(_.p == 'produced_by').alias('event_edges')
site_edges = edges.filter(_.p == 'sampling_site').alias('site_edges')
site_location_edges = edges.filter(_.p == 'site_location').alias('site_location_edges')

samples_via_sites_ibis = (
    samples
    .join(event_edges, samples.row_id == event_edges.s)
    .join(events, event_edges.o[0] == events.row_id)
    .join(site_edges, events.row_id == site_edges.s)
    .join(sites, site_edges.o[0] == sites.row_id)
    .join(site_location_edges, sites.row_id == site_location_edges.s)
    .join(
        locations.filter(_.latitude.notnull()),
        site_location_edges.o[0] == locations.row_id
    )
    .select(
        sample_id=samples.pid,
        sample_label=samples.label,
        site_name=sites.label,
        latitude=locations.latitude,
        longitude=locations.longitude,
        location_type=ibis.literal('via_site_location')
    )
    .limit(100)
)

result_via_sites_ibis = samples_via_sites_ibis.execute()
print(f"Found {len(result_via_sites_ibis)} samples with site-based coordinates (Ibis)")
result_via_sites_ibis.head()

In [ ]:
# Ibis version: get_sample_locations_for_viz function

def get_sample_locations_for_viz_ibis(limit=10000):
    """Extract sample locations optimized for visualization using Ibis"""

    event_edges = edges.filter(_.p == 'produced_by').alias('event_edges')
    sample_location_edges = edges.filter(_.p == 'sample_location').alias('sample_location_edges')
    site_edges = edges.filter(_.p == 'sampling_site').alias('site_edges')
    site_location_edges = edges.filter(_.p == 'site_location').alias('site_location_edges')

    # Direct locations: Sample -> Event -> sample_location -> Location
    direct_locations = (
        samples
        .join(event_edges, samples.row_id == event_edges.s)
        .join(events, event_edges.o[0] == events.row_id)
        .join(sample_location_edges, events.row_id == sample_location_edges.s)
        .join(
            locations.filter((_.latitude.notnull()) & (_.longitude.notnull()) & (~_.obfuscated)),
            sample_location_edges.o[0] == locations.row_id
        )
        .select(
            sample_id=samples.pid,
            label=samples.label,
            latitude=locations.latitude,
            longitude=locations.longitude,
            obfuscated=locations.obfuscated,
            location_type=ibis.literal('direct')
        )
    )

    # Site locations: Sample -> Event -> Site -> site_location -> Location
    site_locations = (
        samples
        .join(event_edges, samples.row_id == event_edges.s)
        .join(events, event_edges.o[0] == events.row_id)
        .join(site_edges, events.row_id == site_edges.s)
        .join(sites, site_edges.o[0] == sites.row_id)
        .join(site_location_edges, sites.row_id == site_location_edges.s)
        .join(
            locations.filter((_.latitude.notnull()) & (_.longitude.notnull()) & (~_.obfuscated)),
            site_location_edges.o[0] == locations.row_id
        )
        .select(
            sample_id=samples.pid,
            label=samples.label,
            latitude=locations.latitude,
            longitude=locations.longitude,
            obfuscated=locations.obfuscated,
            location_type=ibis.literal('via_site')
        )
    )

    return direct_locations.union(site_locations).limit(limit).execute()

# Get visualization-ready data using Ibis
viz_data_ibis = get_sample_locations_for_viz_ibis(5000)
print(f"Prepared {len(viz_data_ibis)} samples for visualization (Ibis version)")
if len(viz_data_ibis) > 0:
    print(f"Coordinate bounds: Lat [{viz_data_ibis.latitude.min():.2f}, {viz_data_ibis.latitude.max():.2f}], "
          f"Lon [{viz_data_ibis.longitude.min():.2f}, {viz_data_ibis.longitude.max():.2f}]")
    print(f"Location types: {viz_data_ibis.location_type.value_counts().to_dict()}")
else:
    print("No samples found with valid coordinates")

viz_data_ibis.head()

### Comparison: Raw SQL vs Ibis

Both approaches implement the same **generic PQG graph traversal patterns**. The Ibis versions offer several advantages:

#### **Readability Benefits:**
1. **Clear separation**: Generic PQG operations (joins on s/p/o) vs OpenContext filters (entity types)
2. **Meaningful aliases**: `samples`, `events`, `locations` make the domain model clear
3. **Method chaining**: Natural Python syntax that reads left-to-right
4. **Type safety**: Ibis can catch column reference errors at definition time

#### **Maintainability Benefits:**
1. **Modular queries**: Easy to swap OpenContext predicates without changing graph traversal logic
2. **Reusable components**: Base table filters separate framework from domain
3. **IDE support**: Auto-completion works for both PQG fields and domain fields
4. **Debugging**: Can inspect intermediate results by executing partial chains

#### **Performance Considerations:**
- Both compile to the same SQL, leveraging DuckDB's query optimizer
- The graph traversal pattern (joining through edges) is the same
- Performance is determined by the underlying PQG structure, not the query interface

In [ ]:
# Quick performance and correctness comparison
import time

print("=== PERFORMANCE COMPARISON ===")

# Time the DuckDB SQL query
perf_conn = duckdb.connect()
perf_conn.execute(f"CREATE VIEW pqg AS SELECT * FROM read_parquet('{parquet_path}');")

start_time = time.time()
sql_result = perf_conn.execute("""
    SELECT COUNT(*) FROM (
        SELECT s.pid as sample_id
        FROM pqg s
        JOIN pqg e1 ON s.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt ON e1.o[1] = evt.row_id
        JOIN pqg e2 ON evt.row_id = e2.s AND e2.p = 'sample_location'
        JOIN pqg g  ON e2.o[1] = g.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND evt.otype = 'SamplingEvent'
          AND g.otype = 'GeospatialCoordLocation'
          AND g.latitude IS NOT NULL
    )
""").fetchone()[0]
sql_time = time.time() - start_time

# Time the Ibis query
start_time = time.time()
ibis_count = samples_with_coords_ibis.count().execute()
ibis_time = time.time() - start_time

print(f"Raw SQL result count: {sql_result}")
print(f"Raw SQL execution time: {sql_time:.3f} seconds")
print(f"Ibis result count: {ibis_count}")
print(f"Ibis execution time: {ibis_time:.3f} seconds")
print(f"Results match: {sql_result == ibis_count}")
print(f"Performance ratio: {ibis_time/sql_time:.2f}x")

perf_conn.close()

print("\n=== KEY TAKEAWAYS ===")
print("✓ Ibis provides much more readable code for complex joins")
print("✓ Performance is comparable (compiles to same SQL)")
print("✓ Good separation of PQG traversal from iSamples semantics")

## Summary

**✅ Fixed Issues:**
- Resolved `AttributeError: 'Table' object has no attribute 'location_edges'` by properly defining aliased edge tables separately
- Fixed duplicate CTE names in the visualization function by using unique aliases
- All Ibis queries now execute successfully

**Key Improvements with Ibis:**
1. **Much cleaner syntax** for multi-step joins - no more cryptic SQL aliases
2. **Step-by-step query building** makes complex logic easier to understand
3. **Reusable components** - define edge tables once, use multiple times
4. **Better debugging** - can inspect intermediate results easily
5. **IDE support** - auto-completion and type checking work better

**Performance:** Ibis compiles to efficient SQL, so performance is equivalent to hand-written queries.

In [ ]:
# Helper function to ensure we have a working DuckDB connection
def ensure_connection():
    """Ensure we have a working DuckDB connection with the parquet view"""
    global conn
    try:
        conn.execute("SELECT 1").fetchone()
    except (NameError, Exception):
        print("Recreating DuckDB connection...")
        conn = duckdb.connect()
        conn.execute(f"CREATE VIEW pqg AS SELECT * FROM read_parquet('{parquet_path}');")
        print("Connection restored!")
    return conn

# Test the connection
ensure_connection()
print("DuckDB connection is ready!")

In [ ]:
def ark_to_url(pid: str) -> str:
    """Return a resolvable n2t.net URL for an ARK identifier.
    If pid is not an ARK, return it as a string.
    """
    if isinstance(pid, str) and pid.startswith("ark:/"):
        return f"https://n2t.net/{pid}"
    return str(pid)

# Quick smoke test if a sample_pid is already in scope (harmless if not)
if 'sample_pid' in globals():
    print("Sample URL:", ark_to_url(sample_pid))

## Utilities

Helper functions used across the notebook (defined early for clarity and reuse).

In [ ]:
# Samples via the site location path for comparison
ensure_connection()

samples_via_sites = conn.execute("""
    SELECT
        s.pid as sample_id,
        s.label as sample_label,
        site.label as site_name,
        g.latitude,
        g.longitude,
        'via_site_location' as location_type
    FROM pqg s
    JOIN pqg e1   ON s.row_id = e1.s AND e1.p = 'produced_by'
    JOIN pqg evt  ON e1.o[1] = evt.row_id
    JOIN pqg e2   ON evt.row_id = e2.s AND e2.p = 'sampling_site'
    JOIN pqg site ON e2.o[1] = site.row_id
    JOIN pqg e3   ON site.row_id = e3.s AND e3.p = 'site_location'
    JOIN pqg g    ON e3.o[1] = g.row_id
    WHERE s.otype = 'MaterialSampleRecord'
      AND evt.otype = 'SamplingEvent'
      AND site.otype = 'SamplingSite'
      AND g.otype = 'GeospatialCoordLocation'
      AND g.latitude IS NOT NULL
    LIMIT 100
""").fetchdf()

print(f"Found {len(samples_via_sites)} samples with site-based coordinates")
samples_via_sites.head()

### Query 2: Trace Samples Through Events to Sites

This demonstrates a more complex **generic PQG traversal pattern** with **OpenContext-specific** archaeological hierarchies.

In [ ]:
# Trace samples through events to sites
sample_site_hierarchy = conn.execute("""
    WITH sample_to_site AS (
        SELECT
            samp.pid as sample_id,
            samp.label as sample_label,
            evt.pid as event_id,
            site.pid as site_id,
            site.label as site_name
        FROM pqg samp
        JOIN pqg e1   ON samp.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt  ON e1.o[1] = evt.row_id AND evt.otype = 'SamplingEvent'
        JOIN pqg e2   ON evt.row_id = e2.s AND e2.p = 'sampling_site'
        JOIN pqg site ON e2.o[1] = site.row_id AND site.otype = 'SamplingSite'
        WHERE samp.otype = 'MaterialSampleRecord'
    )
    SELECT
        site_name,
        COUNT(*) as sample_count
    FROM sample_to_site
    GROUP BY site_name
    ORDER BY sample_count DESC
    LIMIT 20
""").fetchdf()

print("Top sites by sample count:")
print(sample_site_hierarchy)

### Query 3: Explore Material Types and Categories

This query shows how **OpenContext domain concepts** (material classifications) are modeled using the **generic PQG framework**.

In [ ]:
# Explore material types and categories
material_analysis = conn.execute("""
    SELECT
        c.label as material_type,
        c.name as category_name,
        COUNT(DISTINCT s.row_id) as sample_count
    FROM pqg s
    JOIN pqg e ON s.row_id = e.s
    JOIN pqg c ON e.o[1] = c.row_id
    WHERE s.otype = 'MaterialSampleRecord'
      AND e.otype = '_edge_'
      AND e.p = 'has_material_category'
      AND c.otype = 'IdentifiedConcept'
    GROUP BY c.label, c.name
    ORDER BY sample_count DESC
    LIMIT 20
""").fetchdf()

print("Most common material types:")
print(material_analysis)

## Query Performance Tips

These tips apply to both **generic PQG patterns** and **OpenContext-specific** queries:

### Generic PQG Optimization:
1. **Filter edges first**: Use `otype = '_edge_'` early in WHERE clauses
2. **Use array indexing carefully**: `o[1]` for first target in edge arrays
3. **Leverage row_id indexes**: Join on row_id fields for best performance

### OpenContext-Specific Optimization:
1. **Filter by entity type early**: e.g., `otype = 'MaterialSampleRecord'`
2. **Use domain predicates**: Filter edges by specific predicates like `produced_by`
3. **Limit geographic queries**: Add bounds when querying latitude/longitude

### Memory Management for Large Graphs:
- Simple node counts: Fast (<1 second)
- Single-hop edge traversal: Moderate (1-5 seconds)
- Multi-hop graph traversal: Can be slow (5-30 seconds)
- Full graph scans: Avoid without filters

## Visualization Preparation

In [ ]:
def get_sample_locations_for_viz(conn, limit=10000):
    """Extract sample locations optimized for visualization (SQL version)"""
    
    return conn.execute(f"""
        WITH direct_locations AS (
            -- Direct path: Sample -> Event -> sample_location -> Location
            SELECT
                s.pid as sample_id,
                s.label as label,
                g.latitude,
                g.longitude,
                g.obfuscated,
                'direct' as location_type
            FROM pqg s
            JOIN pqg e1   ON s.row_id = e1.s AND e1.p = 'produced_by'
            JOIN pqg evt  ON e1.o[1] = evt.row_id
            JOIN pqg e2   ON evt.row_id = e2.s AND e2.p = 'sample_location'
            JOIN pqg g    ON e2.o[1] = g.row_id
            WHERE s.otype = 'MaterialSampleRecord'
              AND evt.otype = 'SamplingEvent'
              AND g.otype = 'GeospatialCoordLocation'
              AND g.latitude IS NOT NULL
              AND g.longitude IS NOT NULL
        ),
        site_locations AS (
            -- Indirect path: Sample -> Event -> Site -> site_location -> Location
            SELECT
                s.pid as sample_id,
                s.label as label,
                g.latitude,
                g.longitude,
                g.obfuscated,
                'via_site' as location_type
            FROM pqg s
            JOIN pqg e1   ON s.row_id = e1.s AND e1.p = 'produced_by'
            JOIN pqg evt  ON e1.o[1] = evt.row_id
            JOIN pqg e2   ON evt.row_id = e2.s AND e2.p = 'sampling_site'
            JOIN pqg site ON e2.o[1] = site.row_id
            JOIN pqg e3   ON site.row_id = e3.s AND e3.p = 'site_location'
            JOIN pqg g    ON e3.o[1] = g.row_id
            WHERE s.otype = 'MaterialSampleRecord'
              AND evt.otype = 'SamplingEvent'
              AND site.otype = 'SamplingSite'
              AND g.otype = 'GeospatialCoordLocation'
              AND g.latitude IS NOT NULL
              AND g.longitude IS NOT NULL
        )
        SELECT
            sample_id,
            label,
            latitude,
            longitude,
            obfuscated,
            location_type
        FROM (
            SELECT * FROM direct_locations
            UNION ALL
            SELECT * FROM site_locations
        )
        WHERE NOT obfuscated  -- Exclude obfuscated locations for public viz
        LIMIT {limit}
    """).fetchdf()

# Get visualization-ready data
viz_data = get_sample_locations_for_viz(conn, 5000)
print(f"Prepared {len(viz_data)} samples for visualization")
if len(viz_data) > 0:
    print(f"Coordinate bounds: Lat [{viz_data.latitude.min():.2f}, {viz_data.latitude.max():.2f}], "
          f"Lon [{viz_data.longitude.min():.2f}, {viz_data.longitude.max():.2f}]")
    print(f"Location types: {viz_data.location_type.value_counts().to_dict()}")
else:
    print("No samples found with valid coordinates")

## Data Export Options

In [ ]:
def export_site_subgraph(conn, site_name_pattern, output_prefix):
    """Export all data related to a specific site"""
    
    # Find the site
    site_info = conn.execute("""
        SELECT row_id, pid, label
        FROM pqg
        WHERE otype = 'SamplingSite'
        AND label LIKE ?
        LIMIT 1
    """, [f'%{site_name_pattern}%']).fetchdf()
    
    if site_info.empty:
        print(f"No site found matching '{site_name_pattern}'")
        return None
    
    site_row_id = site_info.iloc[0]['row_id']
    print(f"Found site: {site_info.iloc[0]['label']}")
    
    # Get all related entities (simplified version - not recursive)
    related_data = conn.execute("""
        WITH site_related AS (
            -- Get the site itself
            SELECT * FROM pqg WHERE row_id = ?
            
            UNION ALL
            
            -- Get edges from the site
            SELECT * FROM pqg e
            WHERE e.otype = '_edge_' AND e.s = ?
            
            UNION ALL
            
            -- Get entities connected to the site
            SELECT n.* FROM pqg e
            JOIN pqg n ON n.row_id = e.o[1]
            WHERE e.otype = '_edge_' AND e.s = ?
        )
        SELECT * FROM site_related
    """, [site_row_id, site_row_id, site_row_id]).fetchdf()
    
    # Save to parquet
    output_file = f"{output_prefix}_{site_info.iloc[0]['pid']}.parquet"
    related_data.to_parquet(output_file)
    print(f"Exported {len(related_data)} rows to {output_file}")
    
    return related_data

# Example usage (commented out to avoid creating files)
# pompeii_data = export_site_subgraph(conn, "Pompeii", "pompeii_subgraph")

## Data Quality Analysis

In [ ]:
# Check for location data quality
location_quality = conn.execute("""
    SELECT
        CASE 
            WHEN obfuscated THEN 'Obfuscated'
            ELSE 'Precise'
        END as location_type,
        COUNT(*) as count,
        AVG(CASE WHEN latitude IS NOT NULL THEN 1.0 ELSE 0.0 END) * 100 as pct_with_coords
    FROM pqg
    WHERE otype = 'GeospatialCoordLocation'
    GROUP BY location_type
""").fetchdf()

print("Location Data Quality:")
print(location_quality)

In [ ]:
# Check for orphaned nodes (nodes not connected by any edge)
orphan_check = conn.execute("""
    WITH connected_nodes AS (
        SELECT DISTINCT s as row_id FROM pqg WHERE otype = '_edge_'
        UNION
        SELECT DISTINCT unnest(o) as row_id FROM pqg WHERE otype = '_edge_'
    )
    SELECT
        n.otype,
        COUNT(*) as orphan_count
    FROM pqg n
    LEFT JOIN connected_nodes c ON n.row_id = c.row_id
    WHERE n.otype != '_edge_' AND c.row_id IS NULL
    GROUP BY n.otype
""").fetchdf()

print("\nOrphaned Nodes by Type:")
print(orphan_check if not orphan_check.empty else "No orphaned nodes found!")

## Summary Statistics

In [ ]:
# Generate comprehensive summary
summary = conn.execute("""
    WITH stats AS (
        SELECT
            COUNT(*) as total_rows,
            COUNT(DISTINCT pid) as unique_pids,
            COUNT(CASE WHEN otype = '_edge_' THEN 1 END) as edge_count,
            COUNT(CASE WHEN otype != '_edge_' THEN 1 END) as node_count,
            COUNT(DISTINCT CASE WHEN otype != '_edge_' THEN otype END) as entity_types,
            COUNT(DISTINCT p) as relationship_types
        FROM pqg
    )
    SELECT * FROM stats
""").fetchdf()

print("Dataset Summary:")
for col in summary.columns:
    print(f"{col}: {summary[col].iloc[0]:,}")

## Debug: Specific Geo Point Analysis

Testing queries for parquet_cesium.qmd debugging. This section demonstrates:
- **Generic PQG debugging**: How to trace edge connections
- **OpenContext validation**: Verifying archaeological data relationships

In [ ]:
# Debug specific geo location from parquet_cesium.qmd
# This section remains provider-agnostic and uses iSamples model semantics

target_geo_pid = "geoloc_7ea562cce4c70e4b37f7915e8384880c86607729"

print(f"=== Debugging geo location: {target_geo_pid} ===\n")

# 1. Find the geo location record
geo_record = conn.execute("""
    SELECT row_id, pid, otype, latitude, longitude 
    FROM pqg 
    WHERE pid = ? AND otype = 'GeospatialCoordLocation'
""", [target_geo_pid]).fetchdf()

print("1. Geo Location Record:")
if not geo_record.empty:
    print(geo_record.to_dict('records')[0])
    geo_row_id = geo_record.iloc[0]['row_id']
    print(f"   Row ID: {geo_row_id}")
else:
    print("   ❌ Geo location not found!")
    geo_row_id = None

In [ ]:
# 2. Check what edges point to this geo location
if geo_row_id is not None:
    geo_row_id_int = int(geo_row_id)
    edges_to_geo = conn.execute("""
        SELECT s, p, otype as edge_type, pid as edge_pid
        FROM pqg 
        WHERE otype = '_edge_' AND ? = ANY(o)
    """, [geo_row_id_int]).fetchdf()

    print(f"\n2. Edges pointing to this geo location ({len(edges_to_geo)} found):")
    if not edges_to_geo.empty:
        edge_summary = edges_to_geo.groupby('p').size().reset_index()
        edge_summary.columns = ['predicate', 'count']
        print(edge_summary)
        print("\nDetailed edges:")
        for _, edge in edges_to_geo.iterrows():
            print(f"   {edge['p']}: row_id {edge['s']} -> geo location")
    else:
        print("   ❌ No edges point to this geo location!")
else:
    print("\n2. Skipping edge analysis - geo location not found")

In [ ]:
# 3. Direct event samples
if geo_row_id is not None:
    direct_samples = conn.execute("""
        SELECT DISTINCT
            s.pid as sample_id,
            s.label as sample_label,
            s.name as sample_name,
            evt.pid as event_id,
            evt.label as event_label,
            'direct_event_location' as location_path
        FROM pqg s
        JOIN pqg e1  ON s.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt ON e1.o[1] = evt.row_id
        JOIN pqg e2  ON evt.row_id = e2.s AND e2.p = 'sample_location'
        JOIN pqg g   ON e2.o[1] = g.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND evt.otype = 'SamplingEvent'
          AND g.otype = 'GeospatialCoordLocation'
          AND g.pid = ?
        LIMIT 20
    """, [target_geo_pid]).fetchdf()

    print(f"\n3. Direct Event Samples ({len(direct_samples)} found):")
    if not direct_samples.empty:
        print(direct_samples[['sample_id', 'sample_label', 'event_id', 'event_label']].head())
    else:
        print("   ❌ No direct event samples found!")
else:
    print("\n3. Skipping direct samples query - geo location not found")

In [ ]:
# 4. Site-associated samples
if geo_row_id is not None:
    site_samples = conn.execute("""
        SELECT DISTINCT
            s.pid as sample_id,
            s.label as sample_label,
            s.name as sample_name,
            evt.pid as event_id,
            evt.label as event_label,
            site.label as site_name,
            'via_site_location' as location_path
        FROM pqg s
        JOIN pqg e1   ON s.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt  ON e1.o[1] = evt.row_id
        JOIN pqg e2   ON evt.row_id = e2.s AND e2.p = 'sampling_site'
        JOIN pqg site ON e2.o[1] = site.row_id
        JOIN pqg e3   ON site.row_id = e3.s AND e3.p = 'site_location'
        JOIN pqg g    ON e3.o[1] = g.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND evt.otype = 'SamplingEvent'
          AND site.otype = 'SamplingSite'
          AND g.otype = 'GeospatialCoordLocation'
          AND g.pid = ?
        LIMIT 20
    """, [target_geo_pid]).fetchdf()

    print(f"\n4. Site-Associated Samples ({len(site_samples)} found):")
    if not site_samples.empty:
        print(site_samples[['sample_id', 'sample_label', 'site_name', 'event_id']].head())
    else:
        print("   ❌ No site-associated samples found!")
else:
    print("\n4. Skipping site samples query - geo location not found")

In [ ]:
# 5. If we found samples, get detailed metadata for the first sample
all_samples = []
if 'direct_samples' in locals() and not direct_samples.empty:
    all_samples.extend(direct_samples.to_dict('records'))
if 'site_samples' in locals() and not site_samples.empty:
    all_samples.extend(site_samples.to_dict('records'))

if all_samples:
    first_sample = all_samples[0]
    sample_pid = first_sample['sample_id']

    print(f"\n5. Detailed metadata for sample: {sample_pid}")
    print(f"   Resolvable URL: {ark_to_url(sample_pid)}")
    print(f"   Sample label: {first_sample.get('sample_label', 'N/A')}")
    print(f"   Location path: {first_sample.get('location_path', 'N/A')}")

    # Materials for this sample
    materials = conn.execute("""
        SELECT DISTINCT
            mat.pid as material_id,
            mat.label as material_type,
            mat.name as material_category
        FROM pqg s
        JOIN pqg e   ON s.row_id = e.s AND e.p = 'has_material_category'
        JOIN pqg mat ON e.o[1] = mat.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND s.pid = ?
          AND e.otype = '_edge_'
          AND mat.otype = 'IdentifiedConcept'
    """, [sample_pid]).fetchdf()

    print(f"\n   Materials ({len(materials)} found):")
    if not materials.empty:
        for _, mat in materials.iterrows():
            print(f"     - {mat['material_type']} ({ark_to_url(mat['material_id'])})")
    else:
        print("     ❌ No materials found!")

    # Agents responsible for this sample
    agents = conn.execute("""
        SELECT DISTINCT
            agent.pid as agent_id,
            agent.label as agent_name,
            agent.name as agent_role
        FROM pqg s
        JOIN pqg e1    ON s.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt   ON e1.o[1] = evt.row_id
        JOIN pqg e2    ON evt.row_id = e2.s AND e2.p = 'responsibility'
        JOIN pqg agent ON e2.o[1] = agent.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND s.pid = ?
          AND e1.otype = '_edge_'
          AND evt.otype = 'SamplingEvent'
          AND e2.otype = '_edge_'
          AND agent.otype = 'Agent'
        LIMIT 10
    """, [sample_pid]).fetchdf()

    print(f"\n   Responsible Agents ({len(agents)} found):")
    if not agents.empty:
        for _, agent in agents.iterrows():
            print(f"     - {agent['agent_name']} ({ark_to_url(agent['agent_id'])})")
    else:
        print("     ❌ No agents found!")
else:
    print("\n5. No samples found to analyze metadata")

In [ ]:
# 6. Summary of findings for this geo location
print(f"\n=== SUMMARY for {target_geo_pid} ===")
if geo_row_id is not None:
    print(f"✅ Geo location found (row_id: {geo_row_id})")
    print(f"📍 Coordinates: {geo_record.iloc[0]['latitude']}, {geo_record.iloc[0]['longitude']}")

    total_samples = len(all_samples)
    direct_count = len([s for s in all_samples if s.get('location_path') == 'direct_event_location'])
    site_count = len([s for s in all_samples if s.get('location_path') == 'via_site_location'])

    print(f"🔬 Total samples found: {total_samples}")
    print(f"   - Direct event samples: {direct_count}")
    print(f"   - Site-associated samples: {site_count}")

    if total_samples > 0:
        print("✅ Sample metadata retrieval successful!")
    else:
        print("❌ No samples found for this location")
else:
    print("❌ Geo location not found in dataset!")

print(f"\n=== END DEBUG for {target_geo_pid} ===\n")

In [ ]:
# 7. Test with a different geo location that has sample_location edges
sample_location_geos = conn.execute("""
    SELECT g.pid, g.latitude, g.longitude, COUNT(*) as edge_count
    FROM pqg e
    JOIN pqg g ON e.o[1] = g.row_id
    WHERE e.otype = '_edge_'
      AND e.p = 'sample_location'
      AND g.otype = 'GeospatialCoordLocation'
    GROUP BY g.pid, g.latitude, g.longitude
    ORDER BY edge_count DESC
    LIMIT 3
""").fetchdf()

print("=== Testing with geo locations that have direct sample_location edges ===")
print(sample_location_geos)

if not sample_location_geos.empty:
    test_geo_pid = sample_location_geos.iloc[0]['pid']
    print(f"\nTesting direct samples query with: {test_geo_pid}")

    test_direct_samples = conn.execute("""
        SELECT DISTINCT
            s.pid as sample_id,
            s.label as sample_label,
            evt.pid as event_id,
            evt.label as event_label
        FROM pqg s
        JOIN pqg e1  ON s.row_id = e1.s AND e1.p = 'produced_by'
        JOIN pqg evt ON e1.o[1] = evt.row_id
        JOIN pqg e2  ON evt.row_id = e2.s AND e2.p = 'sample_location'
        JOIN pqg g   ON e2.o[1] = g.row_id
        WHERE s.otype = 'MaterialSampleRecord'
          AND evt.otype = 'SamplingEvent'
          AND g.otype = 'GeospatialCoordLocation'
          AND g.pid = ?
        LIMIT 5
    """, [test_geo_pid]).fetchdf()

    print(f"Direct samples found: {len(test_direct_samples)}")
    if not test_direct_samples.empty:
        print("✅ Direct event samples exist")
        print(test_direct_samples[['sample_id', 'sample_label', 'event_id']].head())
    else:
        print("❌ Still no direct event samples found")
else:
    print("❌ No geo locations with sample_location edges found")

## Debug Analysis Results

### Key Findings for parquet_cesium.qmd

1. **Geo Location Structure**: The target geo location `geoloc_7ea562cce4c70e4b37f7915e8384880c86607729` exists in the dataset with correct coordinates.

2. **Sample Association**: This specific location has **1 site-associated sample** but **0 direct event samples**.

3. **Query Validation**: Both query paths work correctly:
   - **Direct path**: `Sample → SamplingEvent → sample_location → GeospatialCoordLocation`
   - **Site path**: `Sample → SamplingEvent → SamplingSite → site_location → GeospatialCoordLocation`

4. **Data Availability**: The dataset contains both types of sample associations, but not every geo location has both types.

### Recommendations for parquet_cesium.qmd

- The JavaScript queries are correctly structured and should work
- Some geo locations may only have site-associated samples (like our test case)
- Consider showing both direct and site-associated samples in the UI
- Add debug logging to identify when no samples are found vs. query errors

In [ ]:
# Analysis complete!
print("\nAnalysis complete!")
print("Note: DuckDB connection remains open for interactive use")

## Read PQG key-value metadata (iSamples generic)

The parquet contains KV metadata describing the iSamples PQG schema (see https://github.com/isamplesorg/pqg). We’ll load the keys `pqg_version`, `pqg_primary_key`, `pqg_node_types`, `pqg_edge_fields`, `pqg_literal_fields` to make the notebook self‑describing and provider‑agnostic.

In [ ]:
# Read PQG key-value metadata using PyArrow (provider-agnostic)
import pyarrow.parquet as pq

try:
    md = pq.read_metadata(parquet_path)
    kv_raw = md.metadata or {}
    # Decode byte keys/values to strings
    kv = { (k.decode() if isinstance(k, (bytes, bytearray)) else str(k)):
           (v.decode() if isinstance(v, (bytes, bytearray)) else str(v))
           for k, v in kv_raw.items() }

    wanted_keys = ["pqg_version", "pqg_primary_key", "pqg_node_types", "pqg_edge_fields", "pqg_literal_fields"]
    selected = {k: kv.get(k) for k in wanted_keys if k in kv}

    print("PQG KV metadata (selected):")
    if selected:
        for k in wanted_keys:
            if k in selected:
                print(f"- {k}: {selected[k][:120]}{'...' if len(selected[k])>120 else ''}")
    else:
        print("No PQG KV metadata keys found in file metadata")
except Exception as e:
    print("Unable to read parquet metadata via PyArrow:", e)